# Movie Theme Extraction and Labeling

The goal of this project is to automate the extraction of labeled themes from movie overviews. These themes can replace the overview as a faster and more consistent way of determining what the movie is about. The defined themes would also allow for faster filtering that would allow the user to find a more exact match based on viewing mood.

In [1]:
import pandas as pd


Data used for this project can be found here https://www.kaggle.com/rounakbanik/the-movies-dataset

We will use the overviews and titles from the movies_metadata.csv

In [284]:
df = pd.read_csv('the-movies-dataset\movies_metadata.csv')

<input>:1: DeprecationWarning: invalid escape sequence \m
<input>:1: DeprecationWarning: invalid escape sequence \m
<input>:1: DeprecationWarning: invalid escape sequence \m
<ipython-input-284-e09f6f83ae3d>:1: DeprecationWarning: invalid escape sequence \m
  df = pd.read_csv('the-movies-dataset\movies_metadata.csv')
C:\Users\adrianbr\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
overviews = df['overview'].apply(str).tolist()
overviews = " ".join(overviews)    #prepare for word_tokenize

# Data Preparation

Names like Alan and Jack are common and will appear influence topic model distributions. Given that names are interchangeable when discussing themes, we will identify and remove them as a part of the data preparation process.

In [10]:
import nltk

In [14]:
tokens = nltk.word_tokenize(overviews)
tokens_pos = nltk.pos_tag(tokens)

### Named Entity Recognition:

Use named entity recognition to detect names in the corpus. The ne_chunk function is used to extract chunks from the tagged pos tokens and will return person, places and organizations. 

In [15]:
ne_tree = nltk.ne_chunk(tokens_pos)

In [17]:
#build list of people to remove
people = []                                 
for subtree in ne_tree.subtrees():
    if subtree.label() == 'PERSON':
        person = []
        for key, value in subtree.leaves():
            person.append(key)
        people.append(person)

In [92]:
#counter gives count of each name 
import collections, operator
people_unique = collections.Counter(map(tuple, people))

Named entity recognition identifies some chunks that are not names. Using the frequency list of people_unique and manual check of the high frequency terms is done to create a list of misidentified entites that should remain in the corpus.

In [101]:
exclude_list = [('Hollywood'),('Christmas','Eve'),('San', 'Francisco'), ('Las', 'Vegas'), ('Santa', 'Claus'),
                ('Christmas',), ('Hollywood',), ('Superman',), ('Academy',), ('Godzilla',), ('Academy', 'Award'),
                ('Buenos', 'Aires'), ('Jesus', 'Christ'),('Pearl', 'Harbor'), ('Louisiana',), ('Sequel',), ('Father',), 
                ('Wealthy',), ('Disney',), ('Count', 'Dracula'),('Los', 'Angeles'), ('Monster', 'High'), ('Brazil',), 
                ('Shaolin',), ('Halloween',), ('Navy','SEALS')
               ]

In [102]:
#return clean list of names minus the misidentified names
people_clean = []
for key,val in sorted(people_unique.items(), key=operator.itemgetter(1), reverse = True):
    if key not in exclude_list:
        people_clean.append(' '.join(key))

## Prep data:

* remove people's names
* remove stop words
* remove punctuation
* lemmatization

In [104]:
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
stops = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

other_stops = ["'s","--","..."]
stops.extend(other_stops)


texts = []
for sent in df['overview'].apply(str):
    text = nltk.word_tokenize(sent)
    text = ['PERSON' if word in people_clean else word for word in text  ]  #replace name with PERSON
    text = [word.lower() for word in text] 
    text = [word for word in text if word not in stops]
    text = [lemmatizer.lemmatize(word) for word in text]
    text = [word for word in text if word not in string.punctuation ] 
    texts.append(text)

In [142]:
#additional cleaning 
more_stops = ["``","'ll","''"]
texts_clean = [[word for word in text if word != 'person'] for text in texts]     #remove identified names
texts_clean = [[word for word in text if word not in more_stops] for text in texts_clean]
texts_clean = [[word for word in text if len(word) > 2] for text in texts_clean]
print(texts_clean[:2])

[['led', 'toy', 'live', 'happily', 'room', 'birthday', 'brings', 'lightyear', 'onto', 'scene', 'losing', 'place', 'heart', 'plot', 'circumstance', 'separate', 'owner', 'duo', 'eventually', 'learns', 'put', 'aside', 'difference'], ['sibling', 'discover', 'enchanted', 'board', 'game', 'open', 'door', 'magical', 'world', 'unwittingly', 'invite', 'adult', 'trapped', 'inside', 'game', 'year', 'living', 'room', 'hope', 'freedom', 'finish', 'game', 'prof', 'risky', 'three', 'find', 'running', 'giant', 'rhinoceros', 'evil', 'monkey', 'terrifying', 'creature']]


## LDA topic modeling

Latent Dirichlet Allocation will be used for topic modeling. 

In [107]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora, models

In [143]:
#build bag of words representation of all documents 
dictionary = corpora.Dictionary(texts_clean)
corpus = [dictionary.doc2bow(text) for text in texts_clean ]

2018-12-15 10:58:57,403 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-12-15 10:58:58,337 : INFO : adding document #10000 to Dictionary(27204 unique tokens: ['aside', 'birthday', 'brings', 'circumstance', 'difference']...)
2018-12-15 10:58:59,208 : INFO : adding document #20000 to Dictionary(41438 unique tokens: ['aside', 'birthday', 'brings', 'circumstance', 'difference']...)
2018-12-15 10:59:00,136 : INFO : adding document #30000 to Dictionary(52721 unique tokens: ['aside', 'birthday', 'brings', 'circumstance', 'difference']...)
2018-12-15 10:59:01,053 : INFO : adding document #40000 to Dictionary(64771 unique tokens: ['aside', 'birthday', 'brings', 'circumstance', 'difference']...)
2018-12-15 10:59:01,563 : INFO : built Dictionary(70362 unique tokens: ['aside', 'birthday', 'brings', 'circumstance', 'difference']...) from 45466 documents (total 1206824 corpus positions)


## 300 topics chosen after multiple iterations and coherence score testing

Choosing the right number of topics is an iterative process. After multiple tests, the number of topics that gave the best results was 300. The coherence score and visualization of the topic models for human readability were part of the evaluation process.

In [144]:
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=300, update_every=1, chunksize=5000, passes=3)

2018-12-15 10:59:12,982 : INFO : using symmetric alpha at 0.0033333333333333335
2018-12-15 10:59:12,986 : INFO : using symmetric eta at 0.0033333333333333335
2018-12-15 10:59:13,006 : INFO : using serial LDA version on this node
2018-12-15 10:59:18,725 : INFO : running online (multi-pass) LDA training, 300 topics, 3 passes over the supplied corpus of 45466 documents, updating model once every 5000 documents, evaluating perplexity every 45466 documents, iterating 50x with a convergence threshold of 0.001000
2018-12-15 10:59:18,729 : INFO : PROGRESS: pass 0, at document #5000/45466
2018-12-15 10:59:52,439 : INFO : merging changes from 5000 documents into a model of 45466 documents
2018-12-15 11:00:08,587 : INFO : topic #74 (0.003): 0.013*"story" + 0.009*"love" + 0.009*"group" + 0.008*"two" + 0.007*"father" + 0.006*"becomes" + 0.006*"trial" + 0.005*"film" + 0.005*"get" + 0.005*"run"
2018-12-15 11:00:08,594 : INFO : topic #227 (0.003): 0.011*"two" + 0.010*"home" + 0.009*"get" + 0.007*"take

2018-12-15 11:03:13,492 : INFO : topic #244 (0.003): 0.050*"convicted" + 0.049*"selling" + 0.020*"fully" + 0.016*"jail" + 0.016*"electricity" + 0.015*"odd" + 0.014*"police" + 0.014*"interest" + 0.014*"rifle" + 0.013*"gate"
2018-12-15 11:03:13,499 : INFO : topic #127 (0.003): 0.091*"enemy" + 0.033*"pleasure" + 0.029*"line" + 0.021*"arrested" + 0.018*"immediately" + 0.015*"behind" + 0.012*"life" + 0.011*"prison" + 0.011*"halt" + 0.010*"cab"
2018-12-15 11:03:13,666 : INFO : topic diff=inf, rho=0.408248
2018-12-15 11:03:13,717 : INFO : PROGRESS: pass 0, at document #35000/45466
2018-12-15 11:03:38,004 : INFO : merging changes from 5000 documents into a model of 45466 documents
2018-12-15 11:03:51,903 : INFO : topic #19 (0.003): 0.163*"gang" + 0.030*"coming" + 0.026*"help" + 0.024*"odds" + 0.023*"deeper" + 0.022*"enlists" + 0.017*"confronts" + 0.017*"script" + 0.016*"leader" + 0.014*"friend"
2018-12-15 11:03:51,906 : INFO : topic #278 (0.003): 0.081*"comic" + 0.042*"front" + 0.035*"travelin

2018-12-15 11:06:44,166 : INFO : topic #141 (0.003): 0.107*"camera" + 0.039*"floor" + 0.039*"captive" + 0.033*"luxury" + 0.028*"fascinating" + 0.025*"coffee" + 0.021*"confusing" + 0.020*"longing" + 0.018*"life" + 0.015*"action"
2018-12-15 11:06:44,171 : INFO : topic #258 (0.003): 0.073*"gain" + 0.056*"financial" + 0.046*"saving" + 0.043*"rent" + 0.025*"proper" + 0.023*"scorpion" + 0.021*"portrayed" + 0.018*"money" + 0.017*"briefly" + 0.016*"make"
2018-12-15 11:06:44,326 : INFO : topic diff=inf, rho=0.300242
2018-12-15 11:06:44,363 : INFO : PROGRESS: pass 1, at document #15000/45466
2018-12-15 11:07:02,127 : INFO : merging changes from 5000 documents into a model of 45466 documents
2018-12-15 11:07:16,239 : INFO : topic #192 (0.003): 0.067*"witness" + 0.060*"wake" + 0.059*"brutal" + 0.047*"morning" + 0.045*"arm" + 0.036*"murder" + 0.031*"investigates" + 0.031*"remember" + 0.029*"locked" + 0.028*"angry"
2018-12-15 11:07:16,241 : INFO : topic #288 (0.003): 0.086*"marry" + 0.074*"road" + 0

2018-12-15 11:10:18,928 : INFO : topic #15 (0.003): 0.131*"quest" + 0.061*"score" + 0.044*"genre" + 0.026*"shoe" + 0.018*"ninja" + 0.018*"cross-country" + 0.017*"completed" + 0.016*"way" + 0.016*"along" + 0.015*"find"
2018-12-15 11:10:19,176 : INFO : topic diff=inf, rho=0.300242
2018-12-15 11:10:19,293 : INFO : PROGRESS: pass 1, at document #45000/45466
2018-12-15 11:10:40,992 : INFO : merging changes from 5000 documents into a model of 45466 documents
2018-12-15 11:10:57,845 : INFO : topic #35 (0.003): 0.157*"hospital" + 0.094*"mental" + 0.084*"appearance" + 0.074*"nearby" + 0.058*"suburban" + 0.047*"institution" + 0.027*"tape" + 0.026*"housewife" + 0.012*"accomplish" + 0.011*"fairly"
2018-12-15 11:10:57,849 : INFO : topic #180 (0.003): 0.135*"attack" + 0.119*"army" + 0.038*"pirate" + 0.032*"attacked" + 0.029*"commander" + 0.022*"voyage" + 0.020*"battle" + 0.020*"invention" + 0.019*"save" + 0.019*"prey"
2018-12-15 11:10:57,855 : INFO : topic #107 (0.003): 0.164*"island" + 0.087*"survi

2018-12-15 11:14:23,333 : INFO : topic #263 (0.003): 0.129*"trial" + 0.102*"russia" + 0.074*"district" + 0.056*"ice" + 0.036*"respected" + 0.027*"alliance" + 0.026*"wreck" + 0.022*"2001" + 0.021*"participant" + 0.020*"ex-boyfriend"
2018-12-15 11:14:23,844 : INFO : topic diff=inf, rho=0.287561
2018-12-15 11:14:24,060 : INFO : PROGRESS: pass 2, at document #25000/45466
2018-12-15 11:14:54,286 : INFO : merging changes from 5000 documents into a model of 45466 documents
2018-12-15 11:15:15,659 : INFO : topic #90 (0.003): 0.176*"complete" + 0.063*"status" + 0.054*"visual" + 0.040*"paranoid" + 0.037*"mechanic" + 0.033*"anthology" + 0.027*"satisfy" + 0.026*"quietly" + 0.026*"sour" + 0.025*"vs."
2018-12-15 11:15:15,664 : INFO : topic #169 (0.003): 0.090*"something" + 0.061*"suspect" + 0.046*"someone" + 0.036*"else" + 0.031*"cousin" + 0.023*"begin" + 0.022*"one" + 0.021*"mistake" + 0.019*"physical" + 0.016*"man"
2018-12-15 11:15:15,670 : INFO : topic #31 (0.003): 0.125*"caught" + 0.057*"ready" 

2018-12-15 11:18:02,837 : INFO : topic #282 (0.003): 0.152*"summer" + 0.077*"spend" + 0.072*"released" + 0.045*"object" + 0.034*"web" + 0.031*"fascinated" + 0.030*"online" + 0.029*"pursuit" + 0.028*"cruel" + 0.027*"winner"
2018-12-15 11:18:02,986 : INFO : topic diff=inf, rho=0.287561


In [146]:
#get coherence score 
from gensim.models.coherencemodel import CoherenceModel
cm = CoherenceModel(model=lda, texts=texts_clean, corpus=corpus, coherence='c_v')
coherence = cm.get_coherence()  
print(coherence)

2018-12-15 11:19:36,196 : INFO : using ParallelWordOccurrenceAccumulator(processes=3, batch_size=64) to estimate probabilities from sliding windows
2018-12-15 11:23:17,776 : INFO : 3 accumulators retrieved from output queue
2018-12-15 11:23:57,353 : INFO : accumulated word occurrence stats for 44438 virtual documents


0.3910022859034032


The pyLDAvis library provides a convenient way to visualize the topics and the distribution of terms in each topic.

In [53]:
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\adrianbr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:972: DeprecationWarning: invalid escape sequence \s
  """
C:\Users\adrianbr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:1186: DeprecationWarning: invalid escape sequence \s
  """
C:\Users\adrianbr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\manifold\t_sne.py:420: DeprecationWarning: invalid escape sequence \s
  """
C:\Users\adrianbr\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


# Seed Labels

Two types of chunk patterns will form the basis of the seed phrases. We are looking to form two word phrases that represent things and terms as described by the corpus. The idea is that themes would be explicitly described during some of the overview creations and that similarity mesures can be used to aplly the common themes as labels.

In [113]:
thing_chunker = "CHUNK: {<NN><NN>}"
term_chunker = "CHUNK: {<JJ.*><NN>}"
thing_parser = nltk.RegexpParser(thing_chunker)
term_parser = nltk.RegexpParser(term_chunker)

In [114]:
thing_tree = thing_parser.parse(tokens_pos)
term_tree = term_parser.parse(tokens_pos)

In [115]:
labels = []
for subtree in thing_tree.subtrees():
    if subtree.label() == 'CHUNK':
        label = []
        for key, value in subtree.leaves():
            label.append(key)
        #print(label)
        labels.append(label)
for subtree in term_tree.subtrees():
    if subtree.label() == 'CHUNK':
        label = []
        for key, value in subtree.leaves():
            label.append(key)
        #print(label)
        labels.append(label)

In [116]:
unique_labels = [list(label) for label in set(tuple(label) for label in labels)]

We can examine the seed list to find potential matches that contain a specific word.

In [369]:
potentials = [label for label in unique_labels if 'club' in label]
potentials

[['chess', 'club'],
 ['ground', 'club'],
 ['bondage', 'club'],
 ['grime', 'club'],
 ['sex', 'club'],
 ['senior', 'club'],
 ['club', 'member'],
 ['club', 'proprietor'],
 ['club', 'scene'],
 ['auto', 'club'],
 ['roll', 'club'],
 ['club', 'kid'],
 ['motorcycle', 'club'],
 ['trendy', 'club'],
 ['volleyball', 'club'],
 ['happy', 'club'],
 ['dance', 'club'],
 ['monster', 'club'],
 ['club', 'life'],
 ['soccer', 'club'],
 ['club', 'championship'],
 ['private', 'club'],
 ['underground', 'club'],
 ['gay', 'club'],
 ['club', 'manager'],
 ['secret', 'club'],
 ['football', 'club'],
 ['bowling', 'club'],
 ['trout', 'club'],
 ['golf', 'club'],
 ['correspondence', 'club'],
 ['Spanish', 'club'],
 ['exclusive', 'club'],
 ['supper', 'club'],
 ['manipulative', 'club'],
 ['gourmet', 'club'],
 ['jazz', 'club'],
 ['club', 'house'],
 ['female', 'club'],
 ['gambling', 'club'],
 ['night', 'club'],
 ['fist', 'club'],
 ['hunting', 'club'],
 ['sumo', 'club'],
 ['club', 'dancer'],
 ['poetry', 'club'],
 ['go-go', 'c

### High potential phrases

Here we will collect the top high potential phrases for each topic. 
1. For each word in the topic model find potentials that have one matching word
2. Convert each matching phrase to a bow model using the same dictionary used for lda training
3. Get topic model for the phrase
4.  Save if topic matches current topic and probability is high 

In [153]:
top_matches = {}
for topic_id, words in lda.print_topics(300):
    top_match = []
    top = 0
    for word, rat in lda.show_topic(topic_id):
        potentials = [label for label in unique_labels if word in label]
        for potential in potentials:
            unseen = dictionary.doc2bow(potential)
            for topic, prob in lda[unseen]:
                if topic ==topic_id and prob > 0.6:
                    if topic in top_matches:
                        top_matches[topic].append(list(potential))
                    else:
                        top_matches.update({topic : [list(potential)]})
                    if prob > top:
                        top = prob
                        top_match = list(potential)
    print('Top Match:',topic_id, top_match)

2018-12-15 11:33:06,408 : INFO : topic #0 (0.003): 0.126*"north" + 0.119*"adult" + 0.092*"hiding" + 0.040*"gambling" + 0.036*"house" + 0.031*"suspected" + 0.027*"blame" + 0.026*"superstar" + 0.021*"wildlife" + 0.020*"family"
2018-12-15 11:33:06,413 : INFO : topic #1 (0.003): 0.198*"search" + 0.122*"missing" + 0.063*"television" + 0.046*"find" + 0.046*"influence" + 0.039*"undercover" + 0.029*"humorous" + 0.023*"corruption" + 0.022*"woman" + 0.015*"design"
2018-12-15 11:33:06,417 : INFO : topic #2 (0.003): 0.151*"military" + 0.102*"turned" + 0.068*"life" + 0.054*"upside" + 0.023*"fifteen" + 0.020*"motorcycle" + 0.020*"imdb" + 0.019*"world" + 0.017*"riding" + 0.017*"work"
2018-12-15 11:33:06,422 : INFO : topic #3 (0.003): 0.094*"stranger" + 0.071*"minister" + 0.061*"grandfather" + 0.039*"spending" + 0.038*"courage" + 0.037*"scheme" + 0.034*"twenty" + 0.032*"two" + 0.030*"family" + 0.027*"visiting"
2018-12-15 11:33:06,432 : INFO : topic #4 (0.003): 0.116*"black" + 0.113*"south" + 0.071*"wh

2018-12-15 11:33:06,617 : INFO : topic #37 (0.003): 0.163*"crew" + 0.145*"might" + 0.105*"video" + 0.096*"ship" + 0.034*"aboard" + 0.030*"green" + 0.028*"willing" + 0.027*"1970" + 0.017*"infected" + 0.015*"badly"
2018-12-15 11:33:06,625 : INFO : topic #38 (0.003): 0.178*"increasingly" + 0.120*"cancer" + 0.053*"1979" + 0.050*"receiving" + 0.040*"fearing" + 0.035*"diagnosed" + 0.023*"alleged" + 0.022*"plantation" + 0.013*"outpost" + 0.013*"life"
2018-12-15 11:33:06,630 : INFO : topic #39 (0.003): 0.118*"original" + 0.103*"space" + 0.078*"earth" + 0.052*"shocking" + 0.039*"bloody" + 0.027*"ceremony" + 0.026*"first" + 0.025*"evolution" + 0.023*"unfinished" + 0.023*"mall"
2018-12-15 11:33:06,642 : INFO : topic #40 (0.003): 0.123*"river" + 0.121*"national" + 0.103*"elite" + 0.057*"forget" + 0.039*"advertising" + 0.027*"kick" + 0.024*"day-to-day" + 0.023*"provocative" + 0.022*"port" + 0.022*"closeted"
2018-12-15 11:33:06,648 : INFO : topic #41 (0.003): 0.112*"dance" + 0.093*"light" + 0.083*"r

2018-12-15 11:33:06,889 : INFO : topic #74 (0.003): 0.152*"story" + 0.117*"tale" + 0.098*"tell" + 0.082*"short" + 0.072*"classic" + 0.042*"used" + 0.041*"brought" + 0.031*"love" + 0.019*"two" + 0.017*"follows"
2018-12-15 11:33:06,894 : INFO : topic #75 (0.003): 0.080*"single" + 0.066*"realize" + 0.043*"'re" + 0.039*"time" + 0.033*"decide" + 0.027*"get" + 0.025*"together" + 0.025*"two" + 0.022*"difficulty" + 0.022*"meet"
2018-12-15 11:33:06,899 : INFO : topic #76 (0.003): 0.244*"past" + 0.125*"future" + 0.044*"dying" + 0.040*"present" + 0.036*"anyone" + 0.035*"blind" + 0.023*"time" + 0.021*"exclusive" + 0.019*"imagination" + 0.018*"intent"
2018-12-15 11:33:06,903 : INFO : topic #77 (0.003): 0.247*"war" + 0.158*"russian" + 0.042*"inhabitant" + 0.023*"atmosphere" + 0.021*"plague" + 0.020*"caused" + 0.020*"peaceful" + 0.019*"anonymous" + 0.019*"american" + 0.017*"supply"
2018-12-15 11:33:06,906 : INFO : topic #78 (0.003): 0.197*"school" + 0.096*"high" + 0.095*"student" + 0.085*"teacher" + 

2018-12-15 11:33:07,109 : INFO : topic #111 (0.003): 0.119*"crazy" + 0.085*"biggest" + 0.070*"insane" + 0.056*"swedish" + 0.046*"broadcast" + 0.046*"inmate" + 0.029*"shift" + 0.027*"founder" + 0.026*"1968" + 0.024*"asylum"
2018-12-15 11:33:07,115 : INFO : topic #112 (0.003): 0.107*"cut" + 0.104*"minute" + 0.085*"among" + 0.083*"box" + 0.070*"discus" + 0.033*"find" + 0.024*"bigger" + 0.019*"android" + 0.017*"sens" + 0.017*"treated"
2018-12-15 11:33:07,119 : INFO : topic #113 (0.003): 0.206*"could" + 0.095*"ever" + 0.035*"constantly" + 0.034*"first" + 0.034*"empty" + 0.034*"finnish" + 0.032*"werewolf" + 0.029*"imagined" + 0.027*"love" + 0.021*"find"
2018-12-15 11:33:07,119 : INFO : topic #114 (0.003): 0.085*"hour" + 0.076*"intimate" + 0.070*"choice" + 0.067*"inspired" + 0.061*"true" + 0.056*"story" + 0.047*"remarkable" + 0.047*"1960s" + 0.045*"blow" + 0.020*"actual"
2018-12-15 11:33:07,123 : INFO : topic #115 (0.003): 0.136*"race" + 0.074*"capture" + 0.055*"underground" + 0.052*"storm" +

2018-12-15 11:33:07,329 : INFO : topic #148 (0.003): 0.108*"ancient" + 0.063*"sinister" + 0.062*"discovery" + 0.053*"walk" + 0.045*"shy" + 0.034*"showing" + 0.034*"religious" + 0.032*"product" + 0.029*"miracle" + 0.027*"belief"
2018-12-15 11:33:07,334 : INFO : topic #149 (0.003): 0.249*"century" + 0.093*"sends" + 0.086*"20th" + 0.046*"report" + 0.042*"grand" + 0.025*"siege" + 0.023*"cape" + 0.021*"named" + 0.020*"knocked" + 0.020*"controversy"
2018-12-15 11:33:07,338 : INFO : topic #150 (0.003): 0.137*"professor" + 0.108*"thousand" + 0.094*"appear" + 0.088*"bird" + 0.053*"coast" + 0.049*"brutally" + 0.025*"startling" + 0.022*"people" + 0.020*"recognize" + 0.015*"substance"
2018-12-15 11:33:07,341 : INFO : topic #151 (0.003): 0.185*"artist" + 0.087*"musician" + 0.036*"strong" + 0.033*"scene" + 0.033*"sexy" + 0.031*"work" + 0.028*"art" + 0.024*"drawing" + 0.024*"remembers" + 0.020*"fittest"
2018-12-15 11:33:07,346 : INFO : topic #152 (0.003): 0.128*"accidentally" + 0.120*"office" + 0.078

...

2018-12-15 11:33:07,774 : INFO : topic #222 (0.003): 0.096*"asks" + 0.071*"thanks" + 0.062*"queen" + 0.061*"value" + 0.053*"setting" + 0.047*"lifestyle" + 0.035*"youngster" + 0.028*"published" + 0.026*"imagine" + 0.025*"drag"
2018-12-15 11:33:07,778 : INFO : topic #223 (0.003): 0.195*"british" + 0.107*"nazi" + 0.090*"stand-up" + 0.041*"paper" + 0.036*"comedian" + 0.033*"topic" + 0.033*"rehabilitation" + 0.030*"rugby" + 0.022*"black-and-white" + 0.021*"self-proclaimed"
2018-12-15 11:33:07,782 : INFO : topic #224 (0.003): 0.159*"christmas" + 0.101*"ghost" + 0.028*"family" + 0.027*"child" + 0.027*"home" + 0.024*"little" + 0.023*"day" + 0.020*"temper" + 0.020*"come" + 0.018*"take"
2018-12-15 11:33:07,782 : INFO : topic #225 (0.003): 0.077*"warrior" + 0.065*"carry" + 0.059*"football" + 0.057*"blue" + 0.050*"season" + 0.041*"fell" + 0.037*"combination" + 0.030*"destined" + 0.029*"saved" + 0.027*"stone"
2018-12-15 11:33:07,786 : INFO : topic #226 (0.003): 0.120*"indian" + 0.113*"area" + 0.046

2018-12-15 11:33:07,961 : INFO : topic #259 (0.003): 0.094*"unexpected" + 0.055*"came" + 0.042*"quiet" + 0.040*"life" + 0.039*"term" + 0.034*"neither" + 0.031*"passionate" + 0.029*"come" + 0.028*"nowhere" + 0.022*"rejected"
2018-12-15 11:33:07,970 : INFO : topic #260 (0.003): 0.124*"gangster" + 0.058*"promise" + 0.054*"infamous" + 0.051*"surface" + 0.039*"program" + 0.033*"super" + 0.029*"framed" + 0.025*"federal" + 0.024*"beneath" + 0.022*"turkish"
2018-12-15 11:33:07,973 : INFO : topic #261 (0.003): 0.133*"law" + 0.087*"serious" + 0.069*"embark" + 0.064*"refuge" + 0.041*"firm" + 0.034*"lust" + 0.034*"type" + 0.025*"faction" + 0.025*"overtaken" + 0.023*"take"
2018-12-15 11:33:07,980 : INFO : topic #262 (0.003): 0.135*"able" + 0.110*"journalist" + 0.096*"sell" + 0.095*"price" + 0.052*"sold" + 0.046*"campaign" + 0.030*"link" + 0.029*"loyalty" + 0.024*"mobster" + 0.017*"smuggling"
2018-12-15 11:33:07,985 : INFO : topic #263 (0.003): 0.140*"russia" + 0.133*"trial" + 0.069*"ice" + 0.057*"d

2018-12-15 11:33:08,183 : INFO : topic #296 (0.003): 0.097*"spy" + 0.071*"using" + 0.066*"apparently" + 0.066*"oil" + 0.060*"phenomenon" + 0.048*"plot" + 0.040*"explain" + 0.038*"assaulted" + 0.031*"establishment" + 0.030*"nefarious"
2018-12-15 11:33:08,188 : INFO : topic #297 (0.003): 0.118*"happy" + 0.058*"happiness" + 0.054*"avoid" + 0.038*"life" + 0.036*"treat" + 0.030*"find" + 0.025*"particularly" + 0.024*"back" + 0.021*"gifted" + 0.021*"never"
2018-12-15 11:33:08,191 : INFO : topic #298 (0.003): 0.116*"planet" + 0.093*"believe" + 0.085*"answer" + 0.064*"weekend" + 0.048*"sensitive" + 0.028*"groom" + 0.025*"irish" + 0.024*"find" + 0.022*"showdown" + 0.020*"mysterious"
2018-12-15 11:33:08,196 : INFO : topic #299 (0.003): 0.094*"social" + 0.076*"thought" + 0.066*"public" + 0.050*"gay" + 0.047*"feeling" + 0.038*"male" + 0.030*"view" + 0.025*"time" + 0.025*"relationship" + 0.022*"men"


Top Match: 0 ['north', 'side']
Top Match: 1 ['woman', 'search']
Top Match: 2 ['military', 'man']
Top Match: 3 ['young', 'stranger']
Top Match: 4 ['black', 'friend']
Top Match: 5 ['darker', 'path']
Top Match: 6 ['good', 'priest']
Top Match: 7 ['various', 'family']
Top Match: 8 ['extraordinary', 'power']
Top Match: 9 ['tension', 'begin']
Top Match: 10 ['elderly', 'man']
Top Match: 11 ['last', 'shot']
Top Match: 12 ['young', 'college']
Top Match: 13 ['dual', 'identity']
Top Match: 14 ['real', 'life']
Top Match: 15 ['new', 'quest']
Top Match: 16 ['prison', 'life']
Top Match: 17 ['modern', 'life']
Top Match: 18 ['old', 'partner']
Top Match: 19 ['gang', 'visit']
Top Match: 20 ['crime', 'thriller']
Top Match: 21 ['month', 'time']
Top Match: 22 ['mysterious', 'street']
Top Match: 23 ['major', 'show']
Top Match: 24 ['service', 'agent']
Top Match: 25 ['japanese', 'way']
Top Match: 26 ['kill', 'time']
Top Match: 27 ['love', 'approach']
Top Match: 28 ['wrong', 'turn']
Top Match: 29 ['recent', 'wor

Top Match: 236 ['love', 'alive']
Top Match: 237 ['drunk', 'man']
Top Match: 238 ['local', 'detective']
Top Match: 239 ['theatre', 'world']
Top Match: 240 ['young', 'history']
Top Match: 241 ['talk', 'show']
Top Match: 242 ['first', 'born']
Top Match: 243 ['possible', 'move']
Top Match: 244 ['odd', 'family']
Top Match: 245 ['magic', 'show']
Top Match: 246 ['young', 'fan']
Top Match: 247 ['tragic', 'family']
Top Match: 248 ['first', 'period']
Top Match: 249 ['new', 'following']
Top Match: 250 ['new', 'baby']
Top Match: 251 ['treasure', 'hidden']
Top Match: 252 ['woman', 'revenge']
Top Match: 253 ['popular', 'twin']
Top Match: 254 ['wild', 'animal']
Top Match: 255 ['dancer', 'fall']
Top Match: 256 ['world', 'drama']
Top Match: 257 ['independent', 'film']
Top Match: 258 ['wealthy', 'friend']
Top Match: 259 ['unexpected', 'friend']
Top Match: 260 ['time', 'gangster']
Top Match: 261 ['local', 'law']
Top Match: 262 ['able', 'help']
Top Match: 263 ['formal', 'trial']
Top Match: 264 ['common', 

# Replace overview with a list of themes

In [217]:
#create Movie class
class Movie:
    def __init__(self, doc_id, title, overview, topics, labels, label_score):
        self.doc_id = doc_id
        self.title = title
        self.overview = overview
        self.topics = topics
        self.labels = dict(labels)
        self.label_score = label_score

In [120]:
#tfidf model to be used to improve similarity scores
tfidf = models.TfidfModel(corpus)

2018-12-15 09:54:47,987 : INFO : collecting document frequencies
2018-12-15 09:54:47,989 : INFO : PROGRESS: processing document #0
2018-12-15 09:54:48,092 : INFO : PROGRESS: processing document #10000
2018-12-15 09:54:48,193 : INFO : PROGRESS: processing document #20000
2018-12-15 09:54:48,338 : INFO : PROGRESS: processing document #30000
2018-12-15 09:54:48,487 : INFO : PROGRESS: processing document #40000
2018-12-15 09:54:48,563 : INFO : calculating IDF weights for 45466 documents and 70968 features (1161978 matrix non-zeros)


## Get a single label for each topic in an overview

1. Get all the topics  above a minimum threshold for each document
2. Get all the high potential phrases for the topic
3. Covert the phrase to a tfidf bow model
4. Compare phrase and document cosine similarity 
5. Keep highest score above a minimum threshold

If no label meets the minimum threshold then we ignore that topic for overview replacement.

In [221]:
from gensim import matutils


def get_movie_labels(doc_id):  
    top_pot_labels = {}
    for topic_id,prob in lda.get_document_topics(corpus[doc_id], minimum_probability=0.03): 
        top_pot_score = 0
        top_pot_label = []
        for pot in top_matches[topic_id]:
            pot_corpus = tfidf[dictionary.doc2bow(pot)]
            #label score
            pot_score = matutils.cossim(tfidf[corpus[doc_id]], pot_corpus)
            if pot_score > top_pot_score and pot_score > 0.05:
                top_pot_label = list(pot)
                top_pot_score = pot_score
        if len(top_pot_label) > 0 : top_pot_labels.update({topic_id : list(top_pot_label)})

    
    top_pot_sent = []
    for key in top_pot_labels:
        top_pot_sent.append(list(top_pot_labels[key]))
    top_pot_sent = [word for label in top_pot_sent for word in label]

    label_score = matutils.cossim(tfidf[corpus[doc_id]], tfidf[dictionary.doc2bow(top_pot_sent)])
    title = df['title'].loc[doc_id]
    overview = df['overview'].loc[doc_id ]
    topics = {}
    for key, value in lda[corpus[doc_id]]:
        topics.update({key : value})
    
    new_movie = Movie(doc_id, title, overview, topics, top_pot_labels, label_score )
    return new_movie

## Build dictionary of movie objects with theme lists and write to csv file

In [223]:
movies = {}
for doc_id in range(len(corpus)):
    movies.update( {doc_id: get_movie_labels(doc_id)})

In [249]:
import csv

with open('movie_labels.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['doc_id', 'title','overview', 'topics', 'labels','label_score' ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    for movie in range(len(movies)):
        try:
            writer.writerow({'doc_id': movies[movie].doc_id, 'title':movies[movie].title, 'overview':movies[movie].overview ,
                         'topics': movies[movie].topics, 'labels': movies[movie].labels , 'label_score': movies[movie].label_score})
        except:
            print(movie, 'failed')

## Build dictionary of topics and their associated documents

Gather the list of documents for each topic. The labels from the movie object is used instead of the full topic distribution. This will increase the accuracy of the filtered list of movies that match multiple topics. 

In [264]:
from collections import defaultdict
topics_docs = defaultdict(list)

for i, doc in enumerate(movies):
    for key in movies[i].labels:
        topics_docs[key].append(i)

## Get global label based on frequency of words in associated documents



1. Get documents associated with topic 
2. Build word frequency list for top x words
3. Covert the phrase to a tfidf bow model
4. Get cosine similarity for high potential phrases and top frequency list
5. Keep the highest score as the top label



In [361]:
def get_top_label(topic_id):
    num_words  = 10       #number of top frequency words to return
    prob_thres = 0.1      #minimum threshold for probability of topic in document
    topic_docs = []

    for doc in topics_docs[topic_id]:
        doc_topics = lda.get_document_topics(corpus[doc], minimum_probability=prob_thres)
        for key, value in doc_topics:
            if  key == topic_id:
                topic_docs.append(corpus[doc])

    freq_dict = {}
    for doc in topic_docs:
        for word, count in doc:
            if word in freq_dict:
                freq_dict[word] += count
            else:
                freq_dict.update({word: count})

    sorted_dict =  sorted(freq_dict.items(), key=operator.itemgetter(1), reverse=True)


    freq_words = []
    for key, value in sorted_dict[:num_words]:
        freq_words.append(dictionary.id2token[key])
    print('Word Frequency:',freq_words)
    freq_corpus = tfidf[dictionary.doc2bow(freq_words)]

    top_score = 0
    top_score_match = []

    for pot in top_matches[topic_id]:
        pot_corpus = tfidf[dictionary.doc2bow(pot)]
        #label score
        pot_score = matutils.cossim(freq_corpus , pot_corpus)
        if pot_score > top_score:
            top_score_match = list(pot)
            top_score = pot_score
            
    return top_score_match

In [340]:
print(get_top_label(56))
print(lda.show_topic(56))
print(top_matches[56])

['life', 'family', 'woman', 'always', 'friend', 'vision', 'young', 'man', 'get', 'one']
['cheerful', 'woman']
[('always', 0.120550446), ('cross', 0.05623309), ('vision', 0.043705042), ('life', 0.039183065), ('loving', 0.026004331), ('charm', 0.02297734), ('wrote', 0.02174493), ('architect', 0.021143453), ('friend', 0.020754363), ('woman', 0.019915445)]
[['cross', 'country'], ['new', 'vision'], ['vision', 'move'], ['end', 'life'], ['stripper', 'life'], ['loving', 'relationship'], ['loving', 'wife'], ['loving', 'look'], ['loving', 'family'], ['enough', 'charm'], ['country', 'charm'], ['open-hearted', 'charm'], ['famous', 'architect'], ['young', 'architect'], ['dumb', 'woman'], ['cheerful', 'woman']]





============================


# Demo Section

============================

In [305]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

def print_movie_details(movie_id): 
    temp_movie = get_movie_labels(movie_id ) 
    print("Title:", temp_movie.title)
    print("Overview:" ,temp_movie.overview)
    pp.pprint(temp_movie.labels)

## Sample document 

use: <br>
print_movie_details(movie_id)<br>
movie_id range from 0 - 45465


In [367]:
print_movie_details(1256)

Title: Field of Dreams
Overview: Ray Kinsella is an Iowa farmer who hears a mysterious voice telling him to turn his cornfield into a baseball diamond. He does, but the voice's directions don't stop -- even after the spirits of deceased ballplayers turn up to play.
{   119: ['mysterious', 'circus'],
    129: ['great', 'spirit'],
    157: ['mysterious', 'voice'],
    161: ['baseball', 'team'],
    176: ['new', 'direction'],
    252: ['mysterious', 'mansion'],
    291: ['local', 'farmer']}


## Browse topics

topic_id ranges from 0 - 299 <br>
get_top_label(topic_id)

In [364]:
browse_topic_id =  45
print('LDA distribution:', lda.show_topic(browse_topic_id))
print('Theme:',get_top_label(browse_topic_id))

LDA distribution: [('club', 0.10398337), ('choose', 0.05524126), ('behavior', 0.050058357), ('owner', 0.048141845), ('halloween', 0.029622752), ('singing', 0.029364608), ('catastrophe', 0.028086297), ('potential', 0.02472276), ('drummer', 0.023873845), ('stunning', 0.023380758)]
Word Frequency: ['club', 'owner', 'love', 'life', 'new', 'night', 'house', 'young', 'meet', 'must']
Theme: ['night', 'club']


## Find intersection of document lists for multiple topics to filter down to a small list movies

topic_id ranges from 0 - 299 <br> 
chain set(topics_docs[topic_id]) together to find the intersection of documents for topics chained

In [368]:
for doc_id in list(set(topics_docs[45]) & set(topics_docs[189]) ):
    print_movie_details(doc_id)
    print("=======================================")

Title: Girls! Girls! Girls!
Overview: Elvis plays Ross Carpenter, a fishing guide/sailor who loves his life out on the sea. When he finds out his boss is retiring to Arizona, he has to find a way to buy the Westwind, a boat that he and his father built. He is also caught between two women: insensitive club singer Robin and sweet Laurel.
{   31: ['woman', 'caught'],
    45: ['club', 'life'],
    57: ['father', 'try'],
    115: ['life', 'boat'],
    125: ['sweet', 'love'],
    132: ['sea', 'travel'],
    189: ['disturbed', 'woman'],
    226: ['tortured', 'woman'],
    283: ['lead', 'singer']}
Title: Homicidal
Overview: The story centers around a murderous scheme to collect a rich inheritance. The object of murder is Miriam Webster, who is to share in the inheritance with her half brother Warren, who lives with his childhood guardian Helga in the mansion where Warren and Mariam grew up. Confined to a wheelchair after recently suffering a stroke, Helga is cared for by her nurse Emily, a st

Title: This Time Around
Overview: We all wish we could change the past. For Mel that day has come. In junior high, Mel and Gabby (Sara Rue) were the biggest geeks. Of course, Mel had a crush on the most popular guy, Drew Hesler (Brian A. Green). When word spreads of Mel’s crush, he plays a cruel joke on her pretending to be interested, only to turn her down in front of the entire school. Eleven years later, our ugly ducklings are now swans and Mel is a PR executive. When Mel is assigned to handle the publicity for a new restaurant, she is shocked to discover that the owner is a handsome young entrepreneur named Drew Hesler.
{   45: ['young', 'owner'],
    78: ['school', 'day'],
    104: ['first', 'crush'],
    116: ['wish', 'come'],
    163: ['new', 'guy'],
    181: ['immigrant', 'day'],
    189: ['young', 'guard'],
    190: ['new', 'year'],
    205: ['new', 'location']}
Title: Poor Pierrot
Overview: One night, Arlequin come to see his lover Colombine. But then Pierrot knocks at the do

Title: Oasis of Fear
Overview: Two young sexually free hippies, Dick (Ray Lovelock) and Ingrid (Ornella Muti) finance their travels by selling naked snaps of Ingrid until their plan is brought to an abrupt end by the Police. Forced on the run the two seek refuge at a seemingly empty isolated large villa. As it turns out the house is inhabited by the middle-aged Barbara (Irene Papas) who invites them in for some potential three-way hanky-panky that soon locks them into something far more twisted and chilling!
{   16: ['isolated', 'house'],
    25: ['naked', 'man'],
    45: ['potential', 'love'],
    70: ['young', 'lawyer'],
    83: ['young', 'lead'],
    84: ['middle-aged', 'man'],
    101: ['young', 'teenage'],
    169: ['young', 'cousin'],
    189: ['twisted', 'life'],
    216: ['large', 'house'],
    230: ['young', 'police'],
    261: ['find', 'refuge'],
    290: ['cold', 'plan']}
Title: La Cage aux folles
Overview: Two gay men living in St. Tropez have their lives turned upside down